In [1]:
import torch
import math
import networkx as nx
from torch_geometric.data import Data, DataLoader
from torch_geometric.data import DataListLoader
import torch_geometric.utils as utils
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.data import InMemoryDataset
import torch_geometric.nn.pool as poolG


In [2]:
ls

BLM-link-list_out.graphml           nochebuena-link-list_out.graphml
COVID19-link-list_out.graphml       primavera-link-list_out.graphml
GraphUB.ipynb                       quarantine-link-list_out.graphml
Marta_Rovira-link-list_out.graphml  santjordi-link-list_out.graphml
Vaga8Nov-link-list_out.graphml


In [3]:
Gmr = nx.read_graphml("Marta_Rovira-link-list_out.graphml")
Gv8 = nx.read_graphml("Vaga8Nov-link-list_out.graphml")
Gnb = nx.read_graphml("nochebuena-link-list_out.graphml")
Gbml = nx.read_graphml("BLM-link-list_out.graphml")
Gcov = nx.read_graphml("COVID19-link-list_out.graphml")
Gpri = nx.read_graphml("primavera-link-list_out.graphml")
Gqua = nx.read_graphml("quarantine-link-list_out.graphml")
Gsj = nx.read_graphml("santjordi-link-list_out.graphml")

In [4]:
nx.info(Gmr)

'Name: \nType: DiGraph\nNumber of nodes: 26832\nNumber of edges: 53285\nAverage in degree:   1.9859\nAverage out degree:   1.9859'

In [5]:
nx.get_node_attributes(Gmr,'community')

{'pepimares': 1,
 'jonathanmartinz': 1,
 'SylviLG': 2,
 'VilaWeb': 1,
 'martarovira': 1,
 'protestona1': 1,
 'LandaTxema': 2,
 'dihanaris': 3,
 'JohnNieve': 3,
 '87JF87': 3,
 'Albert_Rivera': 3,
 'JULIAPACOIRENE': 4,
 'BobEstropajo': 4,
 'tintintalca': 3,
 'AlfonsoRojoPD': 3,
 'benaiges78': 1,
 'soler_toni': 1,
 'annamariasastre': 2,
 'antoniobanos_': 1,
 'vngsergi': 1,
 'Josesegoviapa': 3,
 'JuanGosset': 3,
 'ferranrmdeusbcn': 1,
 'ferranq9': 2,
 'HumphreyDiuQue': 2,
 'Rafa10894276': 1,
 'lluis_llach': 1,
 'montsemaspi': 1,
 'MardraggomezMar': 3,
 'Tonicanto1': 3,
 'jordi_canyas': 3,
 'JosPastr': 3,
 'xmarchmarti': 1,
 'vpartal': 1,
 'biolucho2005': 3,
 'esrafiki': 1,
 'XSalaimartin': 1,
 'jordimarcar': 2,
 'eldiarioes': 1,
 'jesusmarana': 3,
 'MomentsES': 2,
 'oscarcerezal': 3,
 'indpcom': 3,
 'DeOrcajo': 3,
 'ImmaPuig': 2,
 'Lo_Tio_Collons': 2,
 'Gatosombra1': 1,
 'arnaucerda': 1,
 'Europeas_14': 3,
 'gonnassau': 3,
 'RevoluSSSion': 3,
 'Carloscatetolan': 3,
 'Enric_Hernandez': 3,
 

In [6]:
data_marta_rovira = utils.from_networkx(Gmr)
data_vaga = utils.from_networkx(Gv8)
data_nochebuena = utils.from_networkx(Gnb)

In [ ]:
data_blm = utils.from_networkx(Gbml)
data_covid19 = utils.from_networkx(Gcov)
data_primavera = utils.from_networkx(Gpri)
data_quarantine = utils.from_networkx(Gqua)
data_santjordi = utils.from_networkx(Gsj)

In [ ]:
data_blm.edge_index
data_covid19.edge_index
data_primavera.edge_index
data_quarantine.edge_index
data_santjordi.edge_index

In [ ]:
data_blm.edge_index

In [ ]:
data_blm.participation_index

In [ ]:
def give_me_data(data):
    # edge_index = data.edge_index
    x = torch.tensor([[data.dispersion_index[i], data.participation_index[i], 
                       data.global_hubness[i], data.local_hubness[i]] 
                      for i in range(0, len(data.dispersion_index))], dtype=torch.float)
    return Data(x = x, edge_index = data.edge_index)

In [ ]:
data_mr_work = give_me_data(data_marta_rovira)
data_vaga_work = give_me_data(data_vaga)
data_nochebuena_work = give_me_data(data_nochebuena)

In [ ]:
data_blm_work = give_me_data(data_blm)
data_covid19_work = give_me_data(data_covid19)
data_primavera_work = give_me_data(data_primavera)
data_quarantine_work = give_me_data(data_quarantine)
data_santjordi_work = give_me_data(data_santjordi)

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
#from torch.nn import max_pool
from torch_geometric.nn.pool import TopKPooling, SAGPooling, avg_pool,  max_pool
from torch_geometric.nn.glob import Set2Set

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(data.num_node_features, 4)
        #self.max_pool1d = MaxPool1d(1)
        self.conv2 = GCNConv(4, 4)
        self.sag_pool = SAGPooling(4)
        self.conv3 = GCNConv(4,4)
        #self.set2set = Set2Set(4,1)
        self.topk = TopKPooling(4,0.05)
    

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # Pooling here
        # print(type(x))
        x, edge_index, edge_attr, batch, perm, score = self.sag_pool(x, edge_index)
        # max_pool_x(torch.tensor([[i for i in range(0, data_mr_work.num_nodes)]]),x,torch.tensor([[i for i in range(0, data_mr_work.num_nodes)]]))
        # print(type(x))
        # x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, edge_attr, batch, perm, score = self.sag_pool(x, edge_index)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x, edge_index, edge_attr, batch, perm, score = self.topk(x,edge_index)
        x = F.relu(x)
        #cluster = torch.tensor([[0 for i in range(0, x.size()[0])]])
        #print(cluster)
        # x = global_mean_pool(x, torch.tensor([[0 for i in range(0, x.size()[0])]]))
        #x = avg_pool(torch.tensor([[0,0]]),x)
        # x = global_add_pool(x)
        # x = max_pool(cluster, x)
        
        return x
        #return F.log_softmax(x, dim=1)

In [ ]:
def result_print(name, out_x):
    f_dispersion_index = 0.0
    f_participation_index = 0.0
    f_global_hubness = 0.0
    f_local_hubness = 0.0
    final_size = out_x.size()[0]
    
    for item in out_x:
        f_dispersion_index += item[0].detach().numpy()
        f_participation_index += item[1].detach().numpy()
        f_global_hubness += item[2].detach().numpy()
        f_local_hubness += item[3].detach().numpy()

    f_dispersion_index = f_dispersion_index/final_size
    f_participation_index = f_participation_index/final_size
    f_global_hubness = f_global_hubness/final_size
    f_local_hubness = f_local_hubness/final_size

    print("Data: " + str(name) + "\nd_i: " + str(f_dispersion_index) +
    "\tp_i:" + str(f_participation_index) +
    "\tg_h:" + str(f_global_hubness) +
    "\tl_h:" + str(f_local_hubness))


In [ ]:
dict_sources = {"Marta_Rovira": data_mr_work, "Vaga8Nov": data_vaga_work, 
                "Nochebuena": data_nochebuena_work, "BLM": data_blm_work,
                "COVID19": data_covid19_work, "primavera": data_primavera_work,
                "quarantine": data_quarantine_work, "sant_jordi": data_santjordi_work}

for name, data_work in dict_sources.items():
    device = torch.device('cpu')
    model = Net().to(device)
    data = data_work.to(device)
    #optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    #model.train()
    out = model(data)
    result_print(name, out)
    

In [ ]:


#optimizer.zero_grad()

#out[26000]
#y, edge_index, edge_attr, batch, perm, score = out

#y.size()
#out2, edge_index, edge_attr, batch, perm, score = out
#out.size()
#out

In [ ]:

final_size